In [ ]:
import pandas as pd
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline
import IO

In [ ]:
df = IO.load_excel("./data/formatted_examinations.xlsx").dropna()
df.head()

In [ ]:
# Load the Mistral model and tokenizer from Hugging Face
model_name = "jphme/em_german_leo_mistral"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name)

# Create a text generation pipeline
text_generation_pipeline = pipeline("text-generation", model=model, tokenizer=tokenizer, device=0, return_full_text=False)

In [ ]:
val = 194


In [ ]:
few_shot_examples = [
    ("links knie diskr erguss, kein anteromedialer druckschmerz med äußerechts Seite des Kniegelenks, kein druckschmerz kniescheiben, keine meniskuszeichen rechts knie kein erguss, druckschmerz seitlich kniescheibe", 
     "Das linke Knie hat keinen Erguss, und es gibt keine Schmerzen beim Drücken auf der äußerechts Seite des Kniegelenks und Kniescheibe. Das rechte Knie hat auch keinen Erguss, aber an der Seite der Kniescheibe, gibt es Schmerzen beim Drücken."),
    ("rechts knie deutlicher erguss, keine überstreckung des knies, streckdefizit des knies, angemessene beugung des knies, keine rötung", 
     "Das rechte Knie hat einen deutlichen Erguss, es ist nicht möglich das Knie zu überstrecken und auch nicht es komplett zu strecken. Die Beugung des Knies ist nur angemessen möglich. Das Knie ist nicht gerötet."),
]
# Construct the few-shot prompt
instruction = "Du bist ein Patient, formuliere die Beschreibung deiner Symptome aus."
few_shot_prompt = f"{instruction}\n\n" + "\n\n".join([f"USER: {inp}\nASSISTANT: {out}" for inp, out in few_shot_examples])
for idx, row in df[val:].iterrows():
    print(val)
    # Combine the text and instruction
    prompt = f"{few_shot_prompt}\n\nUSER: {row['Formatted']} \n ASSISTANT:"
    # Generate the output
    generated_text = text_generation_pipeline(prompt, max_new_tokens=100)[0]['generated_text']
    output_text = generated_text
    
    df.at[idx, 'generated_text'] = output_text
    df.to_excel("./data/generated_examinations.xlsx", index=False)
    val += 1

In [ ]:
df['generated_text'].head()

In [15]:
val = """flüssiges Gangbild unter Vollbelastung, linkes Knie rom keine Hyperextension des Knies kein Extensionsdefizit des Knies gute Flexion des Knies, Gegenseite Hyperextension des Knies möglich kein Extensionsdefizit des Knies gute Flexion des Knies, diskreter Erguss"""

In [17]:
prompt = f"{few_shot_prompt}\n\nUSER: {val} \n ASSISTANT:"
# Generate the output
generated_text = text_generation_pipeline(prompt, max_new_tokens=200)[0]['generated_text']
print(generated_text)

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


 Unter Vollbelastung zeigt sich ein flüssiges Gangbild. Das linke Knie kann nicht gebeugt werden (Hyperextension), es gibt kein Defizit in der Streckung des Knies. Die Beugung des Knies ist gut möglich. Das rechte Knie kann gebeugt werden (Hyperextension), es gibt kein Defizit in der Streckung des Knies. Die Beugung des Knies ist gut möglich. Es gibt einen diskreten Erguss im linken Knie.
